### HW_lesson_5

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

#module_path = os.path.abspath(os.path.join(os.pardir))
#if module_path not in sys.path:
#    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from  src.utils import prefilter_items
#from src.metrics import precision_at_k

In [91]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [92]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [93]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [94]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [95]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [96]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [97]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Префильтрация

In [98]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [99]:
n_items_train_before = data_train['item_id'].nunique()
n_items_test_before = data_test['item_id'].nunique()
n_users_train_before = data_train['user_id'].nunique()
n_users_test_before = data_test['user_id'].nunique()

data_train_filtered = prefilter_items(data_train, n_popular=5000, item_features=item_features)
data_test_filtered = prefilter_items(data_test, n_popular=5000, item_features=item_features)

n_items_train_after = data_train_filtered['item_id'].nunique()
n_items_test_after = data_test_filtered['item_id'].nunique()
n_users_train_after = data_train_filtered['user_id'].nunique()
n_users_test_after = data_test_filtered['user_id'].nunique()
print('Decreased # items_train from {} to {}'.format(n_items_train_before, n_items_train_after))
print('Decreased # items_test from {} to {}'.format(n_items_test_before, n_items_test_after))
print('Decreased # users_train from {} to {}'.format(n_users_train_before, n_users_train_after))
print('Decreased # users_test from {} to {}'.format(n_users_test_before, n_users_test_after))

Decreased # items_train from 86865 to 5001
Decreased # items_test from 24329 to 5001
Decreased # users_train from 2499 to 2499
Decreased # users_test from 2042 to 2042


### 2. Подготовка датасета

In [100]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

sample_weight = coo_matrix(user_item_matrix)

user_item_matrix.head(2)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

#создадим sparse матрицу для теста
test_sparse_user_item = csr_matrix(test_user_item_matrix).tocsr()

test_user_item_matrix.head(2)

item_id,397896,480014,707683,819063,819112,819304,819308,819330,819518,819765,...,17214946,17214971,17215026,17249614,17290664,17290929,17328876,17330026,17827241,17974316
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
# Создаем словари для перевода индексов

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### 3. Подготовка users и items features

In [103]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
user_feat.shape

(2499, 7)

In [105]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
202291,69.0,MISC SALES TRAN,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,
397896,69.0,KIOSK-GAS,Private,COUPON/MISC ITEMS,GASOLINE-REG UNLEADED,


In [106]:
item_feat.shape

(5001, 6)

In [107]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [108]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
param_grid = {
            "no_components": np.arange(25,55,25),
            "loss": ["bpr"], 
            "learning_rate": np.arange(.01,.1,.05),
            "item_alpha": np.arange (.01,.1,.05),
            "user_alpha": np.arange (.01,.1,.05),
            "epochs": np.arange(10,71,30),
            "random_state":[42],        }

In [119]:
param_grid = {
            "no_components": [20,40,80],
            "loss": ["bpr"], 
            "learning_rate": [0.005, 0.01, 0.02],
            "item_alpha": [0.2, 0.4, 0.6],
            "user_alpha": [0.05, 0.1, 0.2],
            "epochs": [10, 20, 30],
            "random_state":[42]}

In [120]:
from itertools import product
#product позволит собрать все комбинации из значений словаря

def dict_configs(d):
    for comb in product(*d.values()):
        yield dict(zip(d.keys(), comb))

In [121]:
user_features=csr_matrix(user_feat_lightfm.values).tocsr()
item_features=csr_matrix(item_feat_lightfm.values).tocsr()

In [122]:
def grid_search(sparse_matrix_train,
                sample_weight,
               user_features,
               item_features,
                num_threads = 4
               ):
    for hyperparams in dict_configs(param_grid):
        
        num_epochs = hyperparams.pop("epochs") #убираем параметр epochs из словаря и одновременно возвращаем его 
       
        model = LightFM(**hyperparams)
        
        model.fit((sparse_matrix_train > 0) * 1,
                  sample_weight=sample_weight,
                  user_features=user_features,
                  item_features=item_features,
                  epochs=num_epochs, 
                  num_threads=num_threads,
                  verbose=True)
        #Будем подбирать по score на тесте
        score = precision_at_k(model,
                               test_sparse_user_item,
                               user_features=user_features,
                               item_features=item_features,
                               k=5).mean()

        hyperparams["num_epochs"] = num_epochs

        yield score, hyperparams, model

In [123]:

(score, hyperparams, model) = max(grid_search(sparse_user_item,
                                              sample_weight=sample_weight,
                                              user_features=user_features,
                                              item_features=item_features,
                                             ),
                                  key=lambda x: x[0])

print("Best score {} at {}".format(score, hyperparams))


Epoch: 100%|██████████| 30/30 [01:09<00:00,  2.30s/it]


Best score 0.015475025400519371 at {'no_components': 40, 'loss': 'bpr', 'learning_rate': 0.01, 'item_alpha': 0.6, 'user_alpha': 0.1, 'random_state': 42, 'num_epochs': 10}


In [124]:
"""
Результат подбора параметров:
Best score 0.0155 at 
{'no_components': 40, 
'loss': 'bpr', 
'learning_rate': 0.01, 
'item_alpha': 0.6, 
'user_alpha': 0.1, 
'random_state': 42, 
'num_epochs': 10}"""

"\nРезультат подбора параметров:\nBest score 0.0155 at \n{'no_components': 40, \n'loss': 'bpr', \n'learning_rate': 0.01, \n'item_alpha': 0.6, \n'user_alpha': 0.1, \n'random_state': 42, \n'num_epochs': 10}"

In [128]:

model_new = LightFM(no_components = 40,
                    loss = 'bpr',
                    learning_rate = 0.01,
                    item_alpha = 0.6,
                    user_alpha = 0.1,
                    random_state = 42
                   )
                   

In [129]:
model_new.fit((sparse_user_item > 0) * 1,
              sample_weight=coo_matrix(user_item_matrix),
              user_features=user_features,
              item_features=item_features,
              epochs=10, 
              num_threads=4,
              verbose=False)
              

In [130]:
#Рассчитаем precision на трейне и тесте

train_precision = precision_at_k(model_new,
                                 sparse_user_item,
                                 user_features = user_features,
                                 item_features= item_features,
                                 k=5).mean()

test_precision = precision_at_k(model_new,
                                 test_sparse_user_item,
                                 user_features = user_features,
                                 item_features= item_features,
                                 k=5).mean()

train_precision, test_precision

(0.37607047, 0.015475025)

In [131]:
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [132]:
predictions = model_new.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [133]:
data_train_filtered['score'] = predictions

In [134]:
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
                            unique().reset_index()

In [135]:
predict_result

,user_id,item_id
0,1,"[1082185, 865178, 840361, 1104349, 1061203, 82..."
1,2,"[1082185, 12518330, 1113780, 971504, 840361, 8..."
2,3,"[1082185, 6534178, 1008547, 913210, 871440, 11..."
3,4,"[840361, 5568447, 1044078, 1029743, 970760, 85..."
4,5,"[1084591, 962199, 916122, 1126899, 1029743, 10..."
...,...,...
2481,2496,"[1082185, 12518330, 6533765, 6534178, 981760, ..."
2482,2497,"[1082185, 6533765, 6534178, 981760, 854754, 10..."
2483,2498,"[1082185, 868645, 1070538, 6533889, 202291, 87..."
2484,2499,"[1082185, 6534178, 981760, 6533889, 834117, 90..."


In [136]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [137]:
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [138]:
df_result_for_metrics.rename(columns = {'item_id' : 'lightfm_predictions'}, inplace = True)

In [139]:
df_result_for_metrics.head(2)

,user_id,actual,lightfm_predictions
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1082185, 865178, 840361, 1104349, 1061203, 82..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1082185, 6534178, 1008547, 913210, 871440, 11..."


In [140]:
from src.metrics import precision_at_k as precision_selfmade

In [141]:
df_result_for_metrics.apply(lambda row: precision_selfmade(row['lightfm_predictions'], row['actual'],k=5), axis=1).mean()

0.17194662737393585

In [142]:
"""
param_grid = {
            "no_components": [20,40,80],
            "learning_rate": np.arange(.01,.1,.05),
            "loss" : 'bpr',
            "item_alpha": [0.005, 0.01, 0.02],
            "user_alpha": [0.05, 0.1, 0.2],
            "epochs":[10, 20, 30],
        }
        
"""

'\nparam_grid = {\n            "no_components": [20,40,80],\n            "learning_rate": np.arange(.01,.1,.05),\n            "loss" : \'bpr\',\n            "item_alpha": [0.005, 0.01, 0.02],\n            "user_alpha": [0.05, 0.1, 0.2],\n            "epochs":[10, 20, 30],\n        }\n        \n'

In [143]:
"""
precision = 0
for comp in param_grid['no_components']:
    for lr in param_grid['learning_rate']:
        for ia in param_grid['item_alpha']:
            for ua in param_grid['user_alpha']:
                for epoch in param_grid['epochs']:
                    model = LightFM(no_components=comp,
                                    loss=param_grid['loss'],
                                    learning_rate=lr, 
                                    item_alpha=ia,
                                    user_alpha=ua, 
                                    random_state=42)
                    model.fit((sparse_user_item > 0) * 1,
                              sample_weight=coo_matrix(user_item_matrix),
                              user_features=user_features,
                              item_features=item_features,
                              epochs=epoch, 
                              num_threads=20)
                    
                    #считаем precision на тесте
                    precision_score = precision_at_k(model,
                           test_sparse_user_item,                          
                           user_features = user_features,
                           item_features = item_features,
                           k=5).mean()
                   
                    if precision_score > precision:
                        precision = precision_score
                        print(f'Components - {comp}, Loss - {param_grid["loss"]}, Learning_rate - {lr},'  
                          f'Item-alpha - {ia}, User-alpha - {ua},'
                          f'Epochs - {epoch} -- precision: {precision}')
"""

'\nprecision = 0\nfor comp in param_grid[\'no_components\']:\n    for lr in param_grid[\'learning_rate\']:\n        for ia in param_grid[\'item_alpha\']:\n            for ua in param_grid[\'user_alpha\']:\n                for epoch in param_grid[\'epochs\']:\n                    model = LightFM(no_components=comp,\n                                    loss=param_grid[\'loss\'],\n                                    learning_rate=lr, \n                                    item_alpha=ia,\n                                    user_alpha=ua, \n                                    random_state=42)\n                    model.fit((sparse_user_item > 0) * 1,\n                              sample_weight=coo_matrix(user_item_matrix),\n                              user_features=user_features,\n                              item_features=item_features,\n                              epochs=epoch, \n                              num_threads=20)\n                    \n                    #считаем precis